In [1]:
import requests
import json
from tools import load_ustawy

# Zad. 3

In [97]:
analyzer = {
  "settings": {
    "index": {
      "analysis": {
        "analyzer": {
          "synonym_analyzer": {
            "type": "custom",
            "tokenizer": "standard",
            "filter": [ "lowercase", "synonym_graph", "morfologik_stem"]
          }
        },
        "filter": {
          "synonym_graph": {
            "type": "synonym_graph",
            "lenient": 'true',
            "synonyms": ["kpk, kodeks postępowania karnego",
                         "kpc, kodeks postępowania cywilnego",
                         "kk, kodeks karny",
                         "kc, kodeks cywilny"]
          }
        }
      }
    }
  },
  "mappings":{
       "properties":{
          "ustawy_mappings": {
             "type":"text",
             "analyzer":"synonym_analyzer"
         }
      }
   }
}


# Zad 4.

In [98]:
index_url = "http://localhost:9200"+"/ustawy"
header = {"Content-Type": "application/json"}

In [99]:
r = requests.put(index_url, headers=header, data = json.dumps(analyzer))

# Zad. 5

In [10]:
ustawy = load_ustawy()

In [100]:
for idx, ustawa in enumerate(ustawy):
    r = requests.post(index_url+f'/_doc/{idx}', headers=header, data = json.dumps({"text":ustawa, "ustawy_mappings":ustawa}))

In [78]:
r.json()

{'_index': 'ustawy',
 '_type': '_doc',
 '_id': '1177',
 '_version': 3,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 3533,
 '_primary_term': 2}

# Zad.6

In [101]:
query = {
  "query": {
    "match": {
      "ustawy_mappings": "ustawa"
    }
  }
}

In [102]:
r = requests.get(index_url+"/_count", headers=header, data = json.dumps(query))

In [103]:
r.json()

{'count': 1178,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

# Zad 7.

In [179]:
id = [str(idx) for idx, _ in enumerate(ustawy)]

In [181]:
query={
    "ids":id,
    "parameters": {
    "fields":["text", "ustawy_mappings"],
    "term_statistics":"true",},
  }

In [182]:
r = requests.post(index_url+"/_mtermvectors", headers=header, data = json.dumps(query))

In [185]:
for i in r.json()['docs']:
    if 'ustawić' in i['term_vectors']['ustawy_mappings']['terms'].keys():
        print(i['term_vectors']['ustawy_mappings']['terms']['ustawić'])
        break

{'doc_freq': 378, 'ttf': 913, 'term_freq': 3, 'tokens': [{'position': 4604, 'start_offset': 31366, 'end_offset': 31371}, {'position': 5057, 'start_offset': 34581, 'end_offset': 34586}, {'position': 5269, 'start_offset': 36139, 'end_offset': 36144}]}


In [186]:
for i in r.json()['docs']:
    if 'ustaw' in i['term_vectors']['ustawy_mappings']['terms'].keys():
        print(i['term_vectors']['ustawy_mappings']['terms']['ustaw'])
        break

In [130]:
query={
    "analyzer":"synonym_analyzer",
    "text":"ustaw"
}

In [108]:
r2 = requests.get(index_url+"/_analyze", headers=header, data = json.dumps(query))

In [109]:
r2.json()

{'tokens': [{'token': 'ustawa',
   'start_offset': 0,
   'end_offset': 5,
   'type': '<ALPHANUM>',
   'position': 0},
  {'token': 'ustawić',
   'start_offset': 0,
   'end_offset': 5,
   'type': '<ALPHANUM>',
   'position': 0}]}

# Zad.8

In [140]:
query = {
  "query": {
    "match_phrase": {
      "ustawy_mappings": "kodeks postępowania cywilnego"
    }
  }
}

In [141]:
r = requests.get(index_url+"/_count", headers=header, data = json.dumps(query))

In [142]:
r.json()

{'count': 99,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

# Zad. 9

In [206]:
query = {
  "query": {
    "match_phrase": {
      "ustawy_mappings": {
        "query":"wchodzi w życie",
        "slop":2
      },
    }
  }
}

In [207]:
r = requests.get(index_url+"/_count", headers=header, data = json.dumps(query))

In [208]:
r.json()

{'count': 1174,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

# Zad. 10

In [209]:
query = {
    "size": 10,
    "query": {
        "match": {
            "ustawy_mappings": "konstytucja"
        }
    }
}

In [210]:
r = requests.get(index_url+"/_search", headers=header, data = json.dumps(query))

In [213]:
r.json()['hits']['max_score']

6.867639

In [218]:
ids = []
for hit in r.json()['hits']['hits']:
    ids.append(hit['_id'])

In [220]:
import os

In [221]:
ustawy_konstytucja=[]
for idx, filename in enumerate(os.listdir("ustawy/")):
    if str(idx) in ids:
        ustawy_konstytucja.append(filename)

In [222]:
ustawy_konstytucja

['2000_443.txt',
 '1996_350.txt',
 '2001_23.txt',
 '1999_688.txt',
 '2001_1082.txt',
 '1997_681.txt',
 '1997_642.txt',
 '1996_199.txt',
 '1997_604.txt',
 '1997_629.txt']

# Zad.11

In [249]:
query = {
    "size":10,
    "query": {
        "match_phrase": {
            "ustawy_mappings":{
                "query":"konstytucja"}
        }
    },
"highlight": { 
   "fields": {"ustawy_mappings":{"number_of_fragments": 3}}
}}

In [250]:
r = requests.get(index_url+"/_search", headers=header, data = json.dumps(query))

In [252]:
for hit in r.json()['hits']['hits']:
    print(hit["highlight"]["ustawy_mappings"])

['Inicjatywa ustawodawcza w zakresie przedstawienia Zgromadzeniu Narodowemu projektu nowej <em>Konstytucji</em>', 'Do zgłoszenia projektu <em>Konstytucji</em> załącza się wykaz obywateli popierających zgłoszenie, zawierający', 'Zasady, na których opierać się ma <em>Konstytucja</em> mogą być poddane pod referendum. 2.']
['umowy międzynarodowej lub załącznika nie wypełnia przesłanek określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em>', 'okoliczności, a umowa międzynarodowa nie wypełnia przesłanek określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em>', 'Polskiej do ratyfikacji jest dokonywane po uzyskaniu zgody, o której mowa w art. 89 ust. 1 i art. 90 <em>Konstytucji</em>']
['W razie powstania wątpliwości co do zgodności z <em>Konstytucją</em> celów lub zasad działania partii politycznej', 'Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności celów partii politycznej z <em>Konstytucją</em>,', 'Wnioski o stwierdzenie zgodności z <em>Konstytucją</em> celów partii 